In [0]:
from pyspark.sql.functions import col, split, regexp_replace, when, lit, to_date

storage_account_name = "shopee2"
container_name = "shopee-sales2"
file_name = "20240121_shopee_sample_data (1).csv"
output_folder = "shopee_clean"
blob_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{file_name}"
output_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{output_folder}"


In [0]:
spark.version

In [0]:
file_content = dbutils.fs.head(
    blob_path
)
print(type(file_content))

[Truncated to first 65536 bytes]
<class 'str'>


## Check Size of Data

In [0]:
df = spark.read.csv(blob_path, header=True, inferSchema=True)
print(f"Ukuran : {df.count()}")

Ukuran : 375300


## Scheme Data

In [0]:
df.printSchema()

root
 |-- price_ori: string (nullable = true)
 |-- delivery: string (nullable = true)
 |-- item_category_detail: string (nullable = true)
 |-- specification: string (nullable = true)
 |-- title: string (nullable = true)
 |-- w_date: string (nullable = true)
 |-- link_ori: string (nullable = true)
 |-- item_rating: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- idElastic: string (nullable = true)
 |-- price_actual: string (nullable = true)
 |-- sitename: string (nullable = true)
 |-- idHash: string (nullable = true)
 |-- total_rating: string (nullable = true)
 |-- id: string (nullable = true)
 |-- total_sold: string (nullable = true)
 |-- pict_link: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- desc: string (nullable = true)



#### Memisahkan kolom item_category_detail menjadi main, sub, spesifik category

In [0]:
df.select("item_category_detail").distinct().show(truncate = False)

+-----------------------------------------------------------------------------------------------------------+
|item_category_detail                                                                                       |
+-----------------------------------------------------------------------------------------------------------+
|Shopee | Home & Living | Home Decor | Curtains & Blinds                                                    |
| we will solve it.                                                                                         |
|000 hours of Full HD video recording (32GB)4                                                               |
|Shopee | Mobile & Accessories | Others | Others                                                            |
| short-circuiting                                                                                          |
|Shopee | Health & Beauty | Oral Care | Water Flossers                                                      |
| please L

In [0]:
df.select("item_category_detail").show(3, truncate = False)

+-----------------------------------------------------+
|item_category_detail                                 |
+-----------------------------------------------------+
|Shopee | Women's Bags | Shoulder Bags | Shoulder Bags|
|NULL                                                 |
|NULL                                                 |
+-----------------------------------------------------+
only showing top 3 rows


In [0]:
from pyspark.sql.functions import split, col, when, size

split_category = split(col("item_category_detail"), " \\| ")

df_trans = (
    df
    .withColumn("split_arr", split_category)
    .withColumn(
        "main_category",
        when(col("item_category_detail").startswith("Shopee"), 
             when(size(col("split_arr")) > 1, col("split_arr")[1]).otherwise(None)
        )
    )
    .withColumn(
        "sub_category",
        when(col("item_category_detail").startswith("Shopee"), 
             when(size(col("split_arr")) > 2, col("split_arr")[2]).otherwise(None)
        )
    )
    .withColumn(
        "specific_category",
        when(col("item_category_detail").startswith("Shopee"), 
             when(size(col("split_arr")) > 3, col("split_arr")[3]).otherwise(None)
        )
    )
    .drop("split_arr")
)

In [0]:
df_trans.select("main_category", "sub_category", "specific_category").distinct().show(15, truncate = False)

+--------------------+------------------------+----------------------------+
|main_category       |sub_category            |specific_category           |
+--------------------+------------------------+----------------------------+
|Women's Bags        |Handbags                |Set Handbags                |
|Health & Beauty     |OTC Medicine            |Traditional Medications     |
|Health & Beauty     |Fragrances              |For Her                     |
|Home Appliances     |Cooling & Heating       |Fans                        |
|Men Clothes         |Outerwear               |Jackets                     |
|Health & Beauty     |Eye Make Up             |Eyelid Tape & Eyelashes     |
|Baby & Toys         |Bath & Toiletries       |Wipes & Refills             |
|Tickets & Vouchers  |Services                |Others                      |
|Men's Bags & Wallets|Messenger Bags          |Messenger Bags              |
|NULL                |NULL                    |NULL                        |

#### Cleaning & Transform kolom total_rating ke tipe Double

In [0]:
df_trans.select("total_rating").distinct().show(30, truncate = False)

+----------------+
|total_rating    |
+----------------+
|61              |
|103             |
|126             |
|93              |
|20.5k           |
|0               |
|19.4k           |
|NULL            |
|1.8k            |
| Sodium Chloride|
|5.5k            |
|406             |
|335             |
|16k             |
|87              |
|553             |
|237             |
|2               |
|437             |
|6.5k            |
|136             |
|60              |
|630             |
|330             |
|2k              |
|1.3k            |
|301             |
|157             |
|127             |
|8.3k            |
+----------------+
only showing top 30 rows


In [0]:
df_trans.printSchema()

root
 |-- price_ori: string (nullable = true)
 |-- delivery: string (nullable = true)
 |-- item_category_detail: string (nullable = true)
 |-- specification: string (nullable = true)
 |-- title: string (nullable = true)
 |-- w_date: string (nullable = true)
 |-- link_ori: string (nullable = true)
 |-- item_rating: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- idElastic: string (nullable = true)
 |-- price_actual: string (nullable = true)
 |-- sitename: string (nullable = true)
 |-- idHash: string (nullable = true)
 |-- total_rating: string (nullable = true)
 |-- id: string (nullable = true)
 |-- total_sold: string (nullable = true)
 |-- pict_link: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- specific_category: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, ltrim, lower


count_anomali_rating = df_trans.filter(
    ltrim(col("total_rating")).rlike("^\\d+\\.?\\d*[a-zA-Z]") &
    ~ltrim(col("total_rating")).rlike("^\\d+\\.?\\d*[kK]$")
).select("total_rating").count()

print(f"Jumlah Anomali Rating : {count_anomali_rating}")

df_trans.filter(
    ltrim(col("total_rating")).rlike("^\\d+\\.?\\d*[a-zA-Z]") &
    ~ltrim(col("total_rating")).rlike("^\\d+\\.?\\d*[kK]$")
).select("total_rating").show(10, truncate = False)

Jumlah Anomali Rating : 12
+--------------------------------+
|total_rating                    |
+--------------------------------+
|6fdd1d5f13fa24275718738cebd0ccf2|
| 44L                            |
| 11XL                           |
| 24mm                           |
|4537a58b86e49f126abbed0e7a014686|
|0606f61eb41a1a0f7700d40726eed652|
|69067428ce591839f56aa89be98c8d21|
| 24mm                           |
|6ef04cf15ac98e85994007ffec780fff|
|534faaf267b6dbf56f7c81aba0b75eef|
+--------------------------------+
only showing top 10 rows


In [0]:
df_trans = df_trans.withColumn(
    "total_rating",
    when(
        (ltrim(col("total_rating")).rlike("^\\d") & 
         ltrim(col("total_rating")).rlike("[^0-9\\s]") & 
         ~ltrim(col("total_rating")).rlike("^\\d+\\.?\\d*[kK]$")),
        None
    ).otherwise(col("total_rating"))
)

In [0]:
df_trans.filter(col("total_rating").rlike("^\\d+[a-zA-Z]")) \
    .select("total_rating").show(5, truncate=False)

+------------+
|total_rating|
+------------+
|12k         |
|1k          |
|5k          |
|1k          |
|1k          |
+------------+
only showing top 5 rows


In [0]:
df_trans = df_trans.withColumn("total_rating", 
                    when(
                        col("total_rating").rlike("^[0-9]"), 
                        col("total_rating")
                        ).otherwise(None)
                    )

In [0]:
from pyspark.sql.functions import length

"""
Kita lihat data pada `total_rating` yang memiliki data tidak sesuai.
Setelah di-cek data yang berpola (nk) adalah data yang tepat, tetapi data seperti `382632ASD`
merupakan data salah. 
"""

df_trans.filter(length(col("total_rating")) > 6) \
    .select("total_rating") \
    .distinct().show(30, truncate = False)

+------------+
|total_rating|
+------------+
+------------+



In [0]:
"""
Uppercase semua karakter untuk memudahkan filtering.
"""

from pyspark.sql.functions import upper

df_trans = df_trans.withColumn("total_rating", upper(col("total_rating")))

In [0]:
df_trans = df_trans.withColumn(
    "total_rating", 
    when(length(col("total_rating")) > 6, None).otherwise(col("total_rating"))
)

In [0]:
"""
Cek apakah ada nilai yang Null, sehingga bisa diganti.
"""

df_trans.filter((col("total_rating") is None) | (col("total_rating") == "NULL")) \
    .select("total_rating") \
    .distinct() \
    .show(truncate = False)

+------------+
|total_rating|
+------------+
+------------+



In [0]:
df_trans.select("total_rating").distinct().show(30, truncate = False)

+------------+
|total_rating|
+------------+
|61          |
|103         |
|1.9K        |
|3.5K        |
|126         |
|93          |
|21.6K       |
|6.6K        |
|0           |
|NULL        |
|406         |
|335         |
|87          |
|553         |
|237         |
|11.9K       |
|2           |
|2K          |
|437         |
|5.5K        |
|136         |
|8.3K        |
|60          |
|630         |
|330         |
|65K         |
|301         |
|3.2K        |
|157         |
|19.2K       |
+------------+
only showing top 30 rows


In [0]:
df_trans = df_trans.withColumn(
    "total_rating",
    when(col("total_rating").endswith("K"), regexp_replace("total_rating", "K", "").cast("double") * 1000)
    .when(col("total_rating").endswith("M"), regexp_replace("total_rating", "M", "").cast("double") * 1000000)
    .otherwise(col("total_rating"))
    .cast("double")
)

In [0]:
"""
Cek hasil setelah cleansing.
"""

df_trans.select("total_rating").distinct().show(30, truncate = False)

+------------+
|total_rating|
+------------+
|481.0       |
|948.0       |
|97.0        |
|136.0       |
|NULL        |
|237.0       |
|23.0        |
|301.0       |
|174.0       |
|903.0       |
|9700.0      |
|0.0         |
|34.0        |
|2900.0      |
|2700.0      |
|12000.0     |
|21600.0     |
|6100.0      |
|2600.0      |
|18.0        |
|1.0         |
|19400.0     |
|6.0         |
|227.0       |
|25.0        |
|3500.0      |
|18100.0     |
|15.0        |
|157.0       |
|80.0        |
+------------+
only showing top 30 rows


#### Clean & Transform kolom price_ori

In [0]:
df_trans.select("price_ori").distinct().show(10, truncate = False)

+-----------------------------------------------------------------------------------------------------------+
|price_ori                                                                                                  |
+-----------------------------------------------------------------------------------------------------------+
|# Ready Stock                                                                                              |
|Feature1 : mobile phone adapter / usb adapter / otg cable / Type C usb adapter                             |
|Color : Lotus Pink                                                                                         |
|🌟We only sell high quality products                                                                       |
|Feature:100% brand new and high quality.                                                                   |
|    Transfers Data and Charges                                                                             |
|8.70      

In [0]:
"""
Cleansing pada kolom price_ori untuk menghilangkan data yang tidak sesuai dengan mengganti 
nilai string text tersebut menjadi NULL.
"""

df_trans = df_trans.withColumn(
    "price_ori",
    when(
        (ltrim(col("price_ori")).rlike("^\\d") & 
         ltrim(col("price_ori")).rlike("[^0-9\\s]") & 
         ~ltrim(col("price_ori")).rlike("^\\d+\\.?\\d*[kK]$")),
        None
    ).otherwise(col("price_ori"))
)

In [0]:
df_trans.select("price_ori").distinct().show(10, truncate = False)

+-----------------------------------------------------------------------------------------------------------+
|price_ori                                                                                                  |
+-----------------------------------------------------------------------------------------------------------+
|# Ready Stock                                                                                              |
|Feature1 : mobile phone adapter / usb adapter / otg cable / Type C usb adapter                             |
|Color : Lotus Pink                                                                                         |
|🌟We only sell high quality products                                                                       |
|Feature:100% brand new and high quality.                                                                   |
|    Transfers Data and Charges                                                                             |
|Jantina ya

In [0]:
df_trans = df_trans.withColumn(
    "price_ori",
    when(ltrim(col("price_ori")).rlike("^\\d+\\.?\\d*[a-zA-Z]") &
    ~ltrim(col("price_ori")).rlike("^\\d+\\.?\\d*[kK]$"), None).otherwise(col("total_rating"))    
)

In [0]:
df_trans.select("price_ori").distinct().show(10, truncate = False)

+---------+
|price_ori|
+---------+
|481.0    |
|948.0    |
|97.0     |
|136.0    |
|NULL     |
|237.0    |
|23.0     |
|301.0    |
|174.0    |
|903.0    |
+---------+
only showing top 10 rows


In [0]:
df_trans2 = df_trans.withColumn(
    "price_ori", 
    col("price_ori").cast("double")
)

In [0]:
df_trans2.select("price_ori").distinct().show(5, truncate=False)

+---------+
|price_ori|
+---------+
|481.0    |
|948.0    |
|97.0     |
|136.0    |
|NULL     |
+---------+
only showing top 5 rows


#### Clean & Transform price_actual

In [0]:
df_trans2.select("price_actual").distinct().show(30, truncate = False)

+---------------------------------------------+
|price_actual                                 |
+---------------------------------------------+
|8.70                                         |
|29.99                                        |
|1.00                                         |
|179.75                                       |
|19.99                                        |
|0.55                                         |
|0.20                                         |
|198.00                                       |
| and any place where a cool breeze is needed.|
|0.45                                         |
|NULL                                         |
|330.60                                       |
|603.00                                       |
|34.90                                        |
|25.00                                        |
|4.90                                         |
|60.50                                        |
|0.75                                   

In [0]:
df_trans2 = df_trans2.withColumn(
    "price_actual",
    when(
        (ltrim(col("price_actual")).rlike("^\\d") & 
         ltrim(col("price_actual")).rlike("[^0-9\\s]") & 
         ~ltrim(col("price_actual")).rlike("^\\d+\\.?\\d*[kK]$")),
        None
    ).otherwise(col("price_actual"))
)

In [0]:
from pyspark.sql.functions import trim

df_trans2 = df_trans2.withColumn("price_actual", upper(col("price_actual")))

df_trans2 = df_trans2.withColumn(
    "price_actual",
    when(
        trim(col("price_actual")).rlike("^[0-9]+$"),
        trim(col("price_actual"))
    ).otherwise(None)
)

In [0]:
df_trans2.select("price_actual").distinct().show(10, truncate = False)

+------------+
|price_actual|
+------------+
|0           |
|NULL        |
|60          |
|40          |
|44          |
|42          |
|48          |
|47          |
|24          |
|7390        |
+------------+
only showing top 10 rows


In [0]:
df_trans2 = df_trans2.withColumn(
    "price_actual", 
    col("price_actual").cast("double")
)

In [0]:
df_trans2.select("price_actual").distinct().show(10, truncate = False)

+------------+
|price_actual|
+------------+
|NULL        |
|0.0         |
|44.0        |
|40.0        |
|42.0        |
|60.0        |
|47.0        |
|7740.0      |
|7390.0      |
|24.0        |
+------------+
only showing top 10 rows


In [0]:
df_trans2.printSchema()

root
 |-- price_ori: double (nullable = true)
 |-- delivery: string (nullable = true)
 |-- item_category_detail: string (nullable = true)
 |-- specification: string (nullable = true)
 |-- title: string (nullable = true)
 |-- w_date: string (nullable = true)
 |-- link_ori: string (nullable = true)
 |-- item_rating: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- idElastic: string (nullable = true)
 |-- price_actual: double (nullable = true)
 |-- sitename: string (nullable = true)
 |-- idHash: string (nullable = true)
 |-- total_rating: double (nullable = true)
 |-- id: string (nullable = true)
 |-- total_sold: string (nullable = true)
 |-- pict_link: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- specific_category: string (nullable = true)



#### Clean & Transform total_sold

In [0]:
df_trans2.select(
    "total_sold",
).distinct().show(10, truncate = False)  

+-------------+
|total_sold   |
+-------------+
|61           |
|103          |
|126          |
|93           |
|20.5k        |
|0            |
|19.4k        |
|NULL         |
|smilekeychain|
|1.8k         |
+-------------+
only showing top 10 rows


In [0]:
df_trans2 = df_trans2.withColumn(
    "total_sold",
    when(
        (ltrim(col("total_sold")).rlike("^\\d") & 
         ltrim(col("total_sold")).rlike("[^0-9\\s]") & 
         ~ltrim(col("total_sold")).rlike("^\\d+\\.?\\d*[kK]$")),
        None
    ).otherwise(col("total_sold"))
)

In [0]:
df_trans2.select("total_sold").distinct().show(10, truncate = False)

+-------------+
|total_sold   |
+-------------+
|61           |
|103          |
|126          |
|93           |
|20.5k        |
|0            |
|19.4k        |
|NULL         |
|smilekeychain|
|1.8k         |
+-------------+
only showing top 10 rows


In [0]:
df_trans2 = df_trans2.withColumn("total_sold", upper(col("total_sold")))

In [0]:
total_sold_anomali = df_trans2.filter(col("total_sold").rlike("^[A-Z]+$")) \
    .select("total_sold").count()
print(f"Jumlah anomali total_sold: {total_sold_anomali}")    

df_trans2.filter(col("total_sold").rlike("^[A-Z]+$")) \
    .select("total_sold").show(18, truncate=False)

Jumlah anomali total_sold: 16
+-----------------------+
|total_sold             |
+-----------------------+
|SMILEKEYCHAIN          |
|ARABIC                 |
|SMILEKEYCHAIN          |
|SHOPEE                 |
|ALLANTOIN              |
|EZWARE                 |
|INOSITOL               |
|TRIETHOXYCAPRYLYLSILANE|
|WMV                    |
|ARABIC                 |
|SMILEKEYCHAIN          |
|SMILEKEYCHAIN          |
|AGAR                   |
|ALLANTOIN              |
|ARABIC                 |
|TRIETHOXYCAPRYLYLSILANE|
+-----------------------+



In [0]:
df_trans2.filter(length(col("total_sold")) > 5) \
    .select("total_sold").show(10, truncate=False)

+----------------------------------------------------+
|total_sold                                          |
+----------------------------------------------------+
|SMILEKEYCHAIN                                       |
| MAGNESIUM NITRATE                                  |
| ETHYLHEXYLGLYCERIN                                 |
|C465761050E04A218EA69993FE7E4B69                    |
| XANTHAN GUM                                        |
| DRY FUELS                                          |
| MAGNESIUM NITRATE                                  |
| IT CAN TAKE AWAY THE MAKEUP ON THE FACE MORE EASILY|
|ARABIC                                              |
| SODIUM POLYACRYLATE                                |
+----------------------------------------------------+
only showing top 10 rows


In [0]:
from pyspark.sql.functions import regexp_extract

df_trans3 = df_trans2.withColumn(
    "total_sold",
    regexp_extract(trim(col("total_sold")), "([0-9]+\\.?[0-9]*)", 1)
)

In [0]:
df_trans3.select("total_sold").distinct().show(10, truncate=False)

+----------+
|total_sold|
+----------+
|2.3       |
|61        |
|103       |
|1.4       |
|126       |
|93        |
|0         |
|2.2       |
|NULL      |
|406       |
+----------+
only showing top 10 rows


In [0]:
df_trans3.filter(col("total_sold") == "") \
    .select("total_sold").show(10, truncate=False)

+----------+
|total_sold|
+----------+
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
|          |
+----------+
only showing top 10 rows


In [0]:
# df_trans = df_trans.withColumn("total_sold", upper(col("total_sold")))
"""
Cleansing data total_sold
"""

df_trans3 = df_trans3.withColumn(
    "total_sold", 
    when(col("total_sold") == "", None).otherwise(col("total_sold")))


In [0]:
df_trans3.select("total_sold").distinct().show(10, truncate=False)

+----------+
|total_sold|
+----------+
|2.3       |
|61        |
|103       |
|1.4       |
|126       |
|93        |
|0         |
|2.2       |
|NULL      |
|406       |
+----------+
only showing top 10 rows


In [0]:
df_trans3 = df_trans3.withColumn(
    "total_sold", 
    col("total_sold").cast("double")
)

In [0]:
df_trans3.select("total_sold").distinct().show(5, truncate = False)

+----------+
|total_sold|
+----------+
|481.0     |
|948.0     |
|1.2       |
|2.3       |
|97.0      |
+----------+
only showing top 5 rows


In [0]:
df_trans3.printSchema()

root
 |-- price_ori: double (nullable = true)
 |-- delivery: string (nullable = true)
 |-- item_category_detail: string (nullable = true)
 |-- specification: string (nullable = true)
 |-- title: string (nullable = true)
 |-- w_date: string (nullable = true)
 |-- link_ori: string (nullable = true)
 |-- item_rating: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- idElastic: string (nullable = true)
 |-- price_actual: double (nullable = true)
 |-- sitename: string (nullable = true)
 |-- idHash: string (nullable = true)
 |-- total_rating: double (nullable = true)
 |-- id: string (nullable = true)
 |-- total_sold: double (nullable = true)
 |-- pict_link: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- specific_category: string (nullable = true)



#### Clean & Transform item_rating

In [0]:
df_trans3.select("item_rating").distinct().show(20, truncate = False)

+---------------------------------------+
|item_rating                            |
+---------------------------------------+
| cool                                  |
|cutekeychain                           |
| soup                                  |
| Live Streaming                        |
|NULL                                   |
| metal                                 |
| Methylparaben                         |
| outdoors                              |
|4.7                                    |
| dating.                               |
|4.4                                    |
| navigation systems                    |
| Tablet PC or anywhere you can imagine.|
| gamepad                               |
|4.0                                    |
|4.9                                    |
| 5XL                                   |
| Phosphorus                            |
| thank you for your support.           |
| Fresh Green                           |
+---------------------------------

In [0]:
df_trans4 = df_trans3.withColumn(
    "item_rating",
    regexp_extract(trim(col("item_rating")), "([0-9]+\\.?[0-9]*)", 1)
)

In [0]:
df_trans4.filter(length(col("item_rating")) > 4) \
    .select("item_rating").distinct().show(20, truncate = False)

+---------------------+
|item_rating          |
+---------------------+
|35803                |
|802.11               |
|77891                |
|43000                |
|91036                |
|2815.6               |
|19538418772          |
|143018185.22635082191|
|35790                |
|26.04                |
+---------------------+



In [0]:
df_trans4.show(5)

+---------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-----------+------------+--------------------+------------+--------+--------------------+------------+--------------------+----------+--------------------+-------------+-------------+--------------------+-------------+-------------+-----------------+
|price_ori|            delivery|item_category_detail|       specification|               title|    w_date|            link_ori|item_rating| seller_name|           idElastic|price_actual|sitename|              idHash|total_rating|                  id|total_sold|           pict_link|     favorite|    timestamp|                desc|main_category| sub_category|specific_category|
+---------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-----------+------------+--------------------+------------+--------+--------------------+------------+---------------

In [0]:
df_trans4 = df_trans4.drop("idHash")
df_trans4.printSchema()

root
 |-- price_ori: double (nullable = true)
 |-- delivery: string (nullable = true)
 |-- item_category_detail: string (nullable = true)
 |-- specification: string (nullable = true)
 |-- title: string (nullable = true)
 |-- w_date: string (nullable = true)
 |-- link_ori: string (nullable = true)
 |-- item_rating: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- idElastic: string (nullable = true)
 |-- price_actual: double (nullable = true)
 |-- sitename: string (nullable = true)
 |-- total_rating: double (nullable = true)
 |-- id: string (nullable = true)
 |-- total_sold: double (nullable = true)
 |-- pict_link: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- specific_category: string (nullable = true)



In [0]:
df_trans4.select("price_ori", "price_actual", "delivery", "title", "main_category", "sub_category", "specific_category", "id", "idElastic", "w_date").distinct().show(20)

+---------+------------+--------------------+--------------------+--------------------+-------------------+-----------------+--------------------+--------------------+----------+
|price_ori|price_actual|            delivery|               title|       main_category|       sub_category|specific_category|                  id|           idElastic|    w_date|
+---------+------------+--------------------+--------------------+--------------------+-------------------+-----------------+--------------------+--------------------+----------+
|     NULL|        NULL| practical and be...|                NULL|                NULL|               NULL|             NULL|                NULL|                NULL|      NULL|
|     NULL|        NULL|               Black|Gray-Blue(As Pict...|                NULL|               NULL|             NULL|                NULL|                NULL|      NULL|
|     NULL|        NULL|known as Laid Fla...|                NULL|                NULL|               NUL

In [0]:
df_trans4 = df_trans4.drop("item_category_detail")
df_trans4.printSchema()

root
 |-- price_ori: double (nullable = true)
 |-- delivery: string (nullable = true)
 |-- specification: string (nullable = true)
 |-- title: string (nullable = true)
 |-- w_date: string (nullable = true)
 |-- link_ori: string (nullable = true)
 |-- item_rating: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- idElastic: string (nullable = true)
 |-- price_actual: double (nullable = true)
 |-- sitename: string (nullable = true)
 |-- total_rating: double (nullable = true)
 |-- id: string (nullable = true)
 |-- total_sold: double (nullable = true)
 |-- pict_link: string (nullable = true)
 |-- favorite: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- specific_category: string (nullable = true)



## Save to Data Lake Storage Gen 2 

In [0]:
df_final = df_trans4

df_final.write.format("parquet").mode("overwrite").save(output_path)
print("success!!!")

success!!!


/bin/bash: line 1: cd: dbfs: No such file or directory
